In [3]:
# Function to parse the landmark string and return a list of floats
def parse_landmark_string(landmark_str):
    # Split the string by commas, then parse each key-value pair
    landmark_values = []
    for item in landmark_str.split(','):
        try:
            # Split by ':' and take the second element, then strip spaces and convert to float
            value = float(item.split(':')[1].strip())
            landmark_values.append(value)
        except ValueError:
            # Handle cases where the conversion fails (e.g., empty strings or malformed data)
            continue
    return landmark_values

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv('imagePoses.csv')

# Process base64 images if necessary (this step is omitted as it depends on further processing needs)

# # Extract pose landmarks and convert them from lists stored as text to numpy arrays
# import ast  # Import Abstract Syntax Trees module
# # Drop rows where 'poseLandmarks' is NaN
# df = df.dropna(subset=['poseLandmarks'])

# df['poseLandmarks'] = df['poseLandmarks'].apply(ast.literal_eval)  # Convert stringified lists to actual lists
# X = np.array(df['poseLandmarks'].tolist())  # Convert lists to a numpy array
import ast
import numpy as np

# Apply the parsing function to each row in the 'poseLandmarks' column
# Ensure to handle NaN values first as shown previously
df['poseLandmarks'] = df['poseLandmarks'].fillna('[]').apply(parse_landmark_string)

# Convert the list of lists into a numpy array
X = np.array(df['poseLandmarks'].tolist())

# Now, you can proceed with scaling
X_scaled = scaler.fit_transform(X.reshape(X.shape[0], -1))



# Normalize landmarks data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(X.shape[0], -1))  # Reshape for StandardScaler and then transform
X_scaled = X_scaled.reshape(X.shape)  # Reshape back to original shape

# Encode the labels (exercise names) to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['exercise'])

# One-hot encode the labels
y_categorical = to_categorical(y_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Reshape input data for LSTM. LSTM expects data in the shape of [samples, time steps, features].
# Adjust based on the actual structure of your landmarks data. Assuming each landmark frame is a single time step:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], -1))  # Adjusted reshape
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], -1))  # Adjusted reshape

# Define the model architecture
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))  # The output layer with a softmax activation

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.3)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy*100:.2f}%')


IndexError: list index out of range

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv('imagePoses.csv')  # Update the path as needed

#drop the row if the column landmark is emptpy
df = df.dropna(subset=['poseLandmarks'])

# Define a function to extract numerical values from the poseLandmarks string
def extract_landmarks(landmark_str):
    # Initialize an empty list to hold the extracted numerical values
    landmarks = []
    # Split the string into parts based on commas
    parts = landmark_str.split(',')
    # Iterate over each part and extract the numerical value
    for part in parts:
        try:
            # Extract the numerical value after the colon and convert it to float
            value = float(part.split(':')[1])
            landmarks.append(value)
        except (ValueError, IndexError):
            # If conversion fails or the split doesn't work as expected, skip this part
            continue
    return landmarks

# Apply the function to each row in the 'poseLandmarks' column
df['poseLandmarks'] = df['poseLandmarks'].apply(lambda x: extract_landmarks(x) if pd.notnull(x) else [])

# Convert the list of lists into a numpy array for the features
X = np.array(df['poseLandmarks'].tolist())

# Flatten X for StandardScaler
X_flattened = X.reshape(X.shape[0], -1)

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['excercise'])
y_categorical = to_categorical(y_encoded)

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flattened)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Reshape input for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
# Define the model architecture
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=120, batch_size=64, validation_split=0.3)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy*100:.2f}%')


(2112, 1, 132) (2112, 9) (529, 1, 132) (529, 9)
Epoch 1/120
24/24 [==============================] - 6s 55ms/step - loss: 2.1794 - accuracy: 0.1725 - val_loss: 2.1393 - val_accuracy: 0.3533
Epoch 2/120
24/24 [==============================] - 0s 5ms/step - loss: 2.1076 - accuracy: 0.2950 - val_loss: 2.0588 - val_accuracy: 0.3770
Epoch 3/120
24/24 [==============================] - 0s 5ms/step - loss: 2.0168 - accuracy: 0.3640 - val_loss: 1.9436 - val_accuracy: 0.4101
Epoch 4/120
24/24 [==============================] - 0s 5ms/step - loss: 1.8940 - accuracy: 0.3985 - val_loss: 1.8004 - val_accuracy: 0.4148
Epoch 5/120
24/24 [==============================] - 0s 5ms/step - loss: 1.7609 - accuracy: 0.4222 - val_loss: 1.6656 - val_accuracy: 0.4353
Epoch 6/120
24/24 [==============================] - 0s 5ms/step - loss: 1.6318 - accuracy: 0.4567 - val_loss: 1.5457 - val_accuracy: 0.4543
Epoch 7/120
24/24 [==============================] - 0s 7ms/step - loss: 1.5379 - accuracy: 0.4547 - val_

In [23]:
import joblib

model.save('exercise_recognition_model2.keras')
joblib.dump(label_encoder, 'label_encoder2.joblib')
joblib.dump(scaler, 'scaler2.save')

['scaler2.save']